## 외국인지분률

In [1]:
import pandas as pd
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
from pykrx import stock
pd.options.display.float_format = '{:.6f}'.format

In [4]:
cdf = pd.read_csv('data/company_name_and_code.csv', index_col = 0)
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,95570
1,125080,AK홀딩스,6840
2,219097,BGF,27410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,5870
727,362238,휴비스,79980
728,156488,휴스틸,5010
729,103176,흥국화재,540


In [7]:
cdf.loc[:, 'stock_code'] = cdf['stock_code'].astype(str).str.strip().str.zfill(6)
cdf['stock_code']

C:\Users\USER\AppData\Local\Temp\ipykernel_16124\3437129154.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['095570' '006840' '027410' '282330' '138930' '001460' '001040' '079160'
 '000120' '011150' '097950' '000590' '012030' '005830' '016610' '000990'
 '015590' '000210' '007340' '004840' '155660' '069730' '092780' '017940'
 '007700' '114090' '078930' '006360' '001250' '007070' '012630' '039570'
 '089470' '294870' '009540' '267250' '267270' '071970' '010620' '042670'
 '267260' '097230' '014790' '003580' '204320' '060980' '011200' '035000'
 '002460' '298050' '015360' '175330' '234080' '001060' '096760' '008970'
 '105560' '009440' '119650' '092220' '016380' '001390' '033180' '001940'
 '025000' '092230' '000040' '044450' '058850' '058860' '093050' '003550'
 '034220' '051900' '032640' '011070' '066570' '037560' '051910' '079550'
 '006260' '000680' '229640' '108320' '001120' '108670' '023150' '035420'
 '181710' '00

0      095570
1      006840
2      027410
3      282330
4      138930
        ...  
726    005870
727    079980
728    005010
729    000540
730    003280
Name: stock_code, Length: 731, dtype: object

In [6]:
df_foreign_2019 = stock.get_exhaustion_rates_of_foreign_investment("20191230")
df_foreign_2019 = df_foreign_2019.reset_index()
df_foreign_2019 = df_foreign_2019.drop(['상장주식수', '보유수량', '한도수량', '한도소진률'], axis = 1)
df_foreign_2019.columns = ['stock_code', 'foreign_ownership_ratio']
df_foreign_2019

,stock_code,foreign_ownership_ratio
0,000020,5.718750
1,000040,16.390625
2,000050,1.870117
3,000060,13.132812
4,000070,8.273438
...,...,...
911,336370,7.390625
912,33637K,2.560547
913,33637L,0.000000
914,338100,0.330078


In [13]:
df_f_2019 = cdf.merge(right = df_foreign_2019, on = 'stock_code', how = 'left').fillna(0)

In [14]:
df_f_2019.isnull().sum()

corp_code                  0
corp_name                  0
stock_code                 0
foreign_ownership_ratio    0
dtype: int64

In [15]:
def foreign(cdf, start_date = 2019, end_date = 2024):
    result_lst = []
    cdf.loc[:, 'stock_code'] = cdf['stock_code'].astype(str).str.strip().str.zfill(6)
    
    for year in range(start_date, end_date +1):
        if year == 2023:
            df = stock.get_exhaustion_rates_of_foreign_investment(f"{year}1228")
            
        elif year == 2022:
            df = stock.get_exhaustion_rates_of_foreign_investment(f"{year}1229")
            
        else:
            df = stock.get_exhaustion_rates_of_foreign_investment(f"{year}1230")        
        
        df = df.reset_index()
        df = df.drop(['상장주식수', '보유수량', '한도수량', '한도소진률'], axis = 1)
        df.columns = ['stock_code', 'foreign_ownership_ratio']
        
        df = cdf.merge(right = df_foreign_2019, on = 'stock_code', how = 'left').fillna(0)
        
        result_lst.append(df)
        
    return result_lst

In [16]:
foreign_2019, foreign_2020, foreign_2021, foreign_2022, foreign_2023, foreign_2024 = foreign(cdf, 2019, 2024)

In [23]:
foreign_lst = [foreign_2019, foreign_2020, foreign_2021, foreign_2022, foreign_2023, foreign_2024]

for year, df in zip(range(2019, 2025), foreign_lst):
    df.to_csv(f'data/foreign/foreign_ownership_ratio_{year}.csv')